In [ ]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#SDXL で必要なモジュールのインストール
%rm -r /content/diffusers
%cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install -U bitsandbytes optimum-quanto peft tensorflow-metadata transformers scikit-learn ftfy accelerate invisible_watermark safetensors controlnet-aux mediapipe timm



下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [ ]:
from huggingface_hub import login
from google.colab import userdata
HF_LOGIN = userdata.get('HF_LOGIN')
login(HF_LOGIN)

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/colabzenn/colab_fluxdev_sample2/FluxDev_sample2.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

import sys
sys.path.append("/content/diffusers/src")

In [ ]:
#モジュールをimportする
from module.module_flux import FLUX
import time

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [ ]:
#モデルの設定を行う。

config_text = """
[FLUX]
device = auto
n_steps=24
seed=42

;model_mode = t2i
model_mode = i2i

strength = 0.6

;from_single_file = False
;base_model_path = black-forest-labs/FLUX.1-schnell
base_model_path = black-forest-labs/FLUX.1-dev

use_controlnet = False
;controlnet_path = Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro
controlnet_path = Shakker-Labs/FLUX.1-dev-ControlNet-Depth

;control_mode0 = canny
;control_mode1 = tile
control_mode2 = depth
;control_mode3 = blur
;control_mode4 = pose
;control_mode5 = gray
;control_mode6 = lq

;lora_weight_repo0 = alfredplpl/flux.1-dev-modern-anime-lora
;lora_weight_path0 = modern-anime-lora-2.safetensors
;lora_weight_repo1 = aleksa-codes/flux-ghibsky-illustration
;lora_weight_path1 = lora.safetensors
;lora_scale0 = 1.0
;lora_scale1 = 1.0
;trigger_word0 = "modern anime style"
;trigger_word1 = "GHIBSKY style painting"
;lora_weight_repo2 = None
;lora_weight_path2 = ./inputs/QTCanimation_lora_v1-PAseer.safetensors
;lora_scale2 = 1.0
;trigger_word2 = ""

;select_solver = LCM
;select_solver = DPM
;select_solver = Eulera
select_solver = FMEuler

use_karras_sigmas = True
scheduler_algorithm_type = dpmsolver++
solver_order = 2

cfg_scale = 3.5
;cfg_scale = 1.0
width = 832
height = 1216
output_type = pil
aesthetic_score = 6
negative_aesthetic_score = 2.5

save_latent_simple = False
save_latent_overstep = False


"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

生成する画像を決定するプロンプトを指定します。

negative_promptを設定することで，ここに書かれている内容を回避できる。


In [ ]:
#読み上げるプロンプトを設定する。

# not use controlnet
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She is singing ,holding a microphone, wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#use controlnet depth
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#use i2i + lora
main_prompt = """
modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face ,make a peace sign with her left hand, wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
"""



#double lora and anime lora only
#main_prompt = """
#a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage
#"""


#use controlnet canny
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes
#"""

#コントロールネットに入力する参照画像は0から6までの数字を想定（0.png,6.webpなど。数字はcontrolnetのモードと一致させてください）
#参照）コントロールネットのモード　canny (0), tile (1), depth (2), blur (3), pose (4), gray (5), low quality (6).
input_refer_image_folder = "./inputs/controlnet/refer"
#コントロールネットに入力する変換後の参照画像の格納フォルダ。命名規則は上記と同様
output_refer_image_folder = "./inputs/controlnet/refer_prepared"

#image to imageモードを利用する際の画像path
i2i_image_path = "./inputs/imagetoimage/inputs1.png"





In [ ]:
flux = FLUX()
#指定しているControlNetで利用できる"./inputs/refer_prepared/0.png"などがすでにある場合は、下記はコメントアウトしても良い
#flux.prepare_multi_referimage(input_refer_image_folder = input_refer_image_folder,output_refer_image_folder = output_refer_image_folder, low_threshold = 100, high_threshold = 200, noise_level=25, blur_radius=5)



In [ ]:
controlnet_conditioning_scale = [0.5]
temp_strength = 0.6
#temp_strength = None

for i in range(5):
      start = time.time()
      image = flux.generate_image(main_prompt,c_image_path = output_refer_image_folder, i_image_path = i2i_image_path, controlnet_conditioning_scale = controlnet_conditioning_scale, temp_strength = temp_strength)
      print("generate image time: ", time.time()-start)
      image.save("./outputs/FLUX_result_{}.png".format(i))

In [ ]:
from google.colab import runtime
get = 0
if get == 1:
    runtime.unassign()